In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("startup_funding.csv")
df.head()

,Sr No,Date dd/mm/yyyy,Startup Name,Industry Vertical,SubVertical,City Location,Investors Name,InvestmentnType,Amount in USD,Remarks
0,1,09/01/2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN
1,2,13/01/2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN
2,3,09/01/2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN
3,4,02/01/2020,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",NaN
4,5,02/01/2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",NaN


In [3]:
# Dropping `Renarks` column - 90% data is missing
df.drop(columns = ['Remarks'], axis = 0, inplace = True)

In [4]:
# Making `Sr No` as a primary index
df.set_index('Sr No', inplace = True)

In [5]:
# Renaming Column as per our convenience
df.rename(columns = {
    'Date dd/mm/yyyy':'date',
    'Startup Name':'startup',
    'Industry Vertical':'vertical',
    'SubVertical':'subvertical',
    'City  Location':'city',
    'Investors Name':'investors',
    'InvestmentnType':'round',
    'Amount in USD':'amount'
}, inplace = True)

In [6]:
df.head()

,date,startup,vertical,subvertical,city,investors,round,amount
Sr No,,,,,,,,
1,09/01/2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000"
2,13/01/2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394"
3,09/01/2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860"
4,02/01/2020,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000"
5,02/01/2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000"


In [7]:
# There are missing values in amount column - Filling them as 0 
df['amount'] = df['amount'].fillna(value = '0')
# The records where the amount value is menioned by 0 is undisclosed in my pov

In [8]:
df['amount'] = df['amount'].str.replace('undisclosed', '0')
df['amount'] = df['amount'].str.replace('unknown', '0')
df['amount'] = df['amount'].str.replace('Undisclosed', '0')
df['amount'] = df['amount'].str.replace(',', '')

In [9]:
df = df[df['amount'].str.isdigit()] # Because there are some values like -> 14342000+ we cannot able to deal with this data

In [10]:
df['amount'] = df['amount'].astype(np.float64)
df['amount'].head()

Sr No
1    200000000.0
2      8048394.0
3     18358860.0
4      3000000.0
5      1800000.0
Name: amount, dtype: float64

In [11]:
# Converting US dollars to INR cr
df['amount'] = df['amount'].map(lambda X: X * 82.5 / 10000000)

In [12]:
# Converting datatype of date column from object to datetime
df['date'] = pd.to_datetime(df['date'], errors = 'coerce') # because there is lot of uncleaned dateformat present in this column

In [13]:
df.isnull().sum()

date           1746
startup           0
vertical        171
subvertical     925
city            180
investors        24
round             2
amount            0
dtype: int64

In [14]:
df = df.dropna(how = 'any', axis = 0, subset = ['vertical', 'city', 'investors', 'round']) # There are huge amount of missing values in date and subvertical column

---

In [15]:
all_investors = []
def preprocess_investors(investors: str) -> list:
    lst = investors.lower().split(", ")

    # Removing unwanted things
    temp = []
    for investor in lst:
        processed = "".join([c for c in investor if c.isalnum() or c == " "])
        temp.append(processed.strip())
    all_investors.append(temp)

df['investors'].apply(preprocess_investors)
df['investors'] = all_investors
df.head()

,date,startup,vertical,subvertical,city,investors,round,amount
Sr No,,,,,,,,
1,2020-09-01,BYJU’S,E-Tech,E-learning,Bengaluru,[tiger global management],Private Equity Round,1650.000000
2,NaT,Shuttl,Transportation,App based shuttle service,Gurgaon,[susquehanna growth equity],Series C,66.399250
3,2020-09-01,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,[sequoia capital india],Series B,151.460595
4,2020-02-01,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,[vinod khatumal],Pre-series A,24.750000
5,2020-02-01,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,[sprout venture partners],Seed Round,14.850000


In [16]:
def recent_investment(investor: str) -> pd.DataFrame:
    return df[df['investors'].map(lambda x: investor in x)]

recent_investment('sequoia capital india')

,date,startup,vertical,subvertical,city,investors,round,amount
Sr No,,,,,,,,
3,2020-09-01,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,[sequoia capital india],Series B,151.460595
69,2019-11-07,Moglix,E-Commerce,Industrial Tools and Equipments,Singapore,"[composite capital management, sequoia capital...",Series D,495.000000
150,2018-12-10,Blackbuck,Logistics Tech,Largest Trucking Platform,Bengaluru,[sequoia capital india],Private Funding,121.110000
232,2018-07-06,Meesho,Consumer Internet,Online Marketplace For Resellers,Bengaluru,"[sequoia capital india, saif partners, y combi...",Private Equity,94.875000
392,2018-02-01,CleverTap,Technology,Mobile App Analytics,Mumbai,"[recruit holdings, sequoia capital india, acce...",Angel / Seed Funding,0.000000
583,2017-06-09,OncoStem,Healthcare,Medical Diagnostic Solutions,Bangalore,"[sequoia capital india, artiman ventures]",Private Equity,49.500000
587,2017-07-09,OYO Rooms,Consumer Internet,Branded budget hotels marketplace,Gurgaon,"[softbank group, sequoia capital india, lights...",Private Equity,2062.500000
602,NaT,Unacademy,Consumer Internet,Online learning platform,Mumbai,"[sequoia capital india, saif partners, nexus v...",Private Equity,94.875000
665,NaT,Dhruva,Technology,Cloud data protection and management solutions,Pune,"[riverwood capital, sequoia capital india, nex...",Private Equity,660.000000


In [17]:
def biggest_investments(investor: str) -> pd.DataFrame:
    all_investments = recent_investment(investor)
    return all_investments.sort_values(by = 'amount', ascending = False).head()

biggest_investments("sequoia capital india")

,date,startup,vertical,subvertical,city,investors,round,amount
Sr No,,,,,,,,
587,2017-07-09,OYO Rooms,Consumer Internet,Branded budget hotels marketplace,Gurgaon,"[softbank group, sequoia capital india, lights...",Private Equity,2062.50
665,NaT,Dhruva,Technology,Cloud data protection and management solutions,Pune,"[riverwood capital, sequoia capital india, nex...",Private Equity,660.00
69,2019-11-07,Moglix,E-Commerce,Industrial Tools and Equipments,Singapore,"[composite capital management, sequoia capital...",Series D,495.00
1189,2016-02-11,FreshDesk,Consumer Internet,SAAS Based HelpDesk Services,Chennai,"[sequoia capital india, accel partners]",Private Equity,453.75
1301,NaT,DailyHunt,Consumer Internet,Vernacular content mobile app,Bangalore,"[bytedance, arun sarin, matrix partners, sequo...",Private Equity,206.25


In [18]:
def investor_sector_distribution(investor: str):
    all_investments = recent_investment(investor)
    grouped_obj =  all_investments.groupby(by = 'vertical')['amount'].count()
    return ((grouped_obj / grouped_obj.sum()) * 100).round(decimals = 2)
    
investor_sector_distribution("sequoia capital")

vertical
App based Bus Pooling Services                 1.67
Banking Analytics Platform                     1.67
Budget Hotel Accommodation                     1.67
Business Messaging App                         1.67
Clinical Genomics Provider                     1.67
Cloud software solutions                       1.67
Consumer Internet                             18.33
Credit Card Fraud protection solutions         1.67
Digital / Mobile Wallet                        1.67
Doctor Appointment booking app                 1.67
E-Commerce                                     3.33
ECommerce                                      3.33
Education                                      3.33
Ethnic Product eCommerce                       1.67
Fashion Ecommerce store                        1.67
Food & Beverage                                3.33
Food Discovery & Delivery Mobile app           1.67
Global Healthcare products                     1.67
Home Furnishing Solutions                      1.67
Hyp

In [19]:
def investor_stage_distribution(investor: str):
    all_investments = recent_investment(investor)
    grouped_obj =  all_investments.groupby(by = 'round')['amount'].count()
    return ((grouped_obj / grouped_obj.sum()) * 100).round(decimals = 2)
    
investor_stage_distribution("sequoia capital")

round
Private Equity    93.33
Seed Funding       5.00
Series D           1.67
Name: amount, dtype: float64

In [20]:
def investor_city_distribution(investor: str):
    all_investments = recent_investment(investor)
    grouped_obj =  all_investments.groupby(by = 'city')['amount'].count()
    return ((grouped_obj / grouped_obj.sum()) * 100).round(decimals = 2)
    
investor_city_distribution("sequoia capital")

city
Ahmedabad     1.67
Bangalore    35.00
Bengaluru     8.33
Gurgaon      18.33
Gurugram      1.67
Jaipur        1.67
Mumbai       20.00
New Delhi    13.33
Name: amount, dtype: float64

In [21]:
# Year on Year investment
def yoy_investment(investor: str):
    all_investments = recent_investment(investor)
    grouped_obj =  all_investments.groupby(by = all_investments['date'].dt.year)['amount'].sum().to_frame()
    grouped_obj.index = grouped_obj.index.astype(np.int64)
    return grouped_obj

yoy_investment("sequoia capital india")

,amount
date,
2016,552.750000
2017,2194.500000
2018,215.985000
2019,495.000000
2020,151.460595


---

In [22]:
# Total Amount Raised
df['amount'].sum().round(decimals = 2)

296496.99

In [23]:
# Max Amount Raised
df['amount'].max().round(decimals = 2)

32175.0

In [24]:
# The company who raised largest amount among all the startups
df[df['amount'] == df['amount'].max()].reset_index()['startup'][0]

'Rapido Bike Taxi'

In [25]:
# Avg amount Raised
(df['amount'].sum() / df['startup'].nunique()).round(decimals = 2)

128.91

In [26]:
df.head()

,date,startup,vertical,subvertical,city,investors,round,amount
Sr No,,,,,,,,
1,2020-09-01,BYJU’S,E-Tech,E-learning,Bengaluru,[tiger global management],Private Equity Round,1650.000000
2,NaT,Shuttl,Transportation,App based shuttle service,Gurgaon,[susquehanna growth equity],Series C,66.399250
3,2020-09-01,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,[sequoia capital india],Series B,151.460595
4,2020-02-01,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,[vinod khatumal],Pre-series A,24.750000
5,2020-02-01,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,[sprout venture partners],Seed Round,14.850000


In [27]:
df['startup'].value_counts().sort_values()

startup
Splitkart                  1
New Castle Technologies    1
My Forex Eye               1
TruxApp                    1
CroFarm                    1
                          ..
UrbanClap                  6
Medinfi                    6
Meesho                     6
Ola Cabs                   8
Swiggy                     8
Name: count, Length: 2300, dtype: int64

In [28]:
df['amount'].describe()

count     2823.000000
mean       105.029045
std        853.305503
min          0.000000
25%          0.000000
50%          4.125000
75%         33.000000
max      32175.000000
Name: amount, dtype: float64

In [29]:
# Month by Month chart of total amount and total funding counts
def mom_total_funding():
    # Creating year and month column
    temp_df = df.copy()
    temp_df['year'] = df['date'].dt.year
    temp_df['month'] = df['date'].dt.month

    # Grouping
    grouped_obj = temp_df.groupby(by = ['year', 'month'], sort = True)
    total_funding = grouped_obj['amount'].sum().to_frame()
    count_funding = grouped_obj['amount'].count()

    total_funding['count'] = count_funding
    return total_funding

mom_tf = mom_total_funding()
mom_tf   

amount  count
year   month                    
2015.0 1.0    5274.637500     35
       2.0    1567.178250     14
       3.0    1910.658750     36
       4.0    1936.646250     21
       5.0     817.863750     22
...                   ...    ...
2019.0 11.0   1153.350000      5
       12.0   1233.375000      5
2020.0 2.0      39.600000      2
       9.0    1801.460595      2
       10.0   1237.500000      1

[62 rows x 2 columns]

In [30]:
# Sector Analysis Pie -> top sectors(Count + Sum)
(df.groupby(by = 'round')['amount'].sum() / df['amount'].sum()) * 100

round
Angel                           0.000403
Angel / Seed Funding            0.003714
Angel Funding                   0.000000
Angel Round                     0.000889
Bridge Round                    0.000000
Corporate Round                 0.105957
Debt Funding                    0.396005
Debt and Preference capital     0.017588
Debt-Funding                    0.013912
Equity                          0.281032
Equity Based Funding            0.086257
Funding Round                   2.782490
Inhouse Funding                 0.006799
Maiden Round                    0.012521
Mezzanine                       0.002782
Pre Series A                    0.000000
Pre-Series A                    0.021904
Pre-series A                    0.008347
Private                         0.043685
Private Equity                 69.015200
Private Equity Round            1.407940
Private Funding                 0.040847
PrivateEquity                   0.041737
Seed                            0.146915
Seed / Ang

In [31]:
(df.groupby(by = 'round')['amount'].count() / df.shape[0]) * 100

round
Angel                           0.035423
Angel / Seed Funding            0.283386
Angel Funding                   0.035423
Angel Round                     0.035423
Bridge Round                    0.035423
Corporate Round                 0.070847
Debt Funding                    0.814736
Debt and Preference capital     0.035423
Debt-Funding                    0.035423
Equity                          0.070847
Equity Based Funding            0.035423
Funding Round                   0.035423
Inhouse Funding                 0.035423
Maiden Round                    0.035423
Mezzanine                       0.035423
Pre Series A                    0.035423
Pre-Series A                    0.141693
Pre-series A                    0.035423
Private                         0.035423
Private Equity                 44.668792
Private Equity Round            0.141693
Private Funding                 0.035423
PrivateEquity                   0.035423
Seed                            0.141693
Seed / Ang

In [32]:
df.head()

,date,startup,vertical,subvertical,city,investors,round,amount
Sr No,,,,,,,,
1,2020-09-01,BYJU’S,E-Tech,E-learning,Bengaluru,[tiger global management],Private Equity Round,1650.000000
2,NaT,Shuttl,Transportation,App based shuttle service,Gurgaon,[susquehanna growth equity],Series C,66.399250
3,2020-09-01,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,[sequoia capital india],Series B,151.460595
4,2020-02-01,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,[vinod khatumal],Pre-series A,24.750000
5,2020-02-01,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,[sprout venture partners],Seed Round,14.850000


In [40]:
temp_grouped = df.groupby(by = [df['date'].dt.year, df['date'].dt.month])
temp_grouped.agg({
    'startup': ['count', 'sum'],
    'investors': ['count']
})

startup                                                     \
              count                                                sum   
date   date                                                              
2015.0 1.0       35  CollegeDekhoBlack White Orange BrandsBiteclubP...   
       2.0       14  RoomsToniteKaaryahTwiglyCityFloShopalystHolaCh...   
       3.0       36  AnakageEasyKhaanaShuttlEckovationFitcircleFind...   
       4.0       21  ApplopCaRPMNDTV Wedding portalSplitkartShirsa ...   
       5.0       22  BookMyBaiMyIndianStayBobbleNykaaVoyllaIndiaLen...   
...             ...                                                ...   
2019.0 11.0       5               Rein GameseBikeGoMoglixSetuCleverTap   
       12.0       5                 EcozenSuperGamingMeeshoMedlifeAzah   
2020.0 2.0        2                 https://www.wealthbucket.in/Fashor   
       9.0        2                                    BYJU’SMamaearth   
       10.0       1                                             Zomato   

            investors  
                count  
date   date            
2015.0 1.0         35  
       2.0         14  
       3.0         36  
       4.0         21  
       5.0         22  
...               ...  
2019.0 11.0         5  
       12.0         5  
2020.0 2.0          2  
       9.0          2  
       10.0         1  

[62 rows x 3 columns]

In [47]:
temp_grouped['amount'].mean().unstack().stack()

date    date
2015.0  1.0      150.703929
        2.0      111.941304
        3.0       53.073854
        4.0       92.221250
        5.0       37.175625
                   ...     
2019.0  11.0     230.670000
        12.0     246.675000
2020.0  2.0       19.800000
        9.0      900.730298
        10.0    1237.500000
Length: 62, dtype: float64

In [ ]:
temp_dict = {}
for (year, month), df in temp_grouped:
    temp_dict[(year, month)] = df

{(2015.0,
  1.0):             date                    startup  \
 Sr No                                         
 2109  2015-01-12               CollegeDekho   
 2110  2015-01-12  Black White Orange Brands   
 2111  2015-01-12                   Biteclub   
 2112  2015-01-12            Patel Logistics   
 2113  2015-01-12                      Wigzo   
 2115  2015-01-12            Godof Hospitals   
 2290  2015-01-10                     Indifi   
 2291  2015-01-10                       Wydr   
 2292  2015-01-10                    EasyFix   
 2293  2015-01-10                     Muvizz   
 2294  2015-01-10                    Abhibus   
 2295  2015-01-10                   Cleartax   
 2383  2015-01-09                      TOFlo   
 2384  2015-01-09                FXMartIndia   
 2385  2015-01-09               Stylecracker   
 2386  2015-01-09                 Luxuryhues   
 2471  2015-01-08                   Snapdeal   
 2472  2015-01-08                      Ressy   
 2569  2015-01-07      